In [1]:
import cv2
import face_recognition
from simple_facerec import SimpleFacerec
import os
import time
import winsound
import threading

# Initialize face recognition system and encode known faces from a specified folder
faces_folder = "G:\Ai\CV\Face Recognition\Face detection Open CV\FACES"
sfr = SimpleFacerec()
sfr.load_encoding_images(faces_folder)

# Open a connection to the webcam
cap = cv2.VideoCapture(0)

# Initialize variables for tracking unknown faces and video recording
unknown_faces_data = []
last_unknown_face_time = 0
cooldown_interval = 5  # Time interval (in seconds) to avoid frequent logging of unknown faces
video_recording = False
video_writer = None
record_start_time = None
video_folder = "unknown_faces_videos"

def play_alert_sound():
    winsound.PlaySound("alert.wav", winsound.SND_FILENAME)




# Ensure the folder for saving unknown faces' videos exists
if not os.path.exists(video_folder):
    os.makedirs(video_folder)

# Start processing frames from the webcam
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Detect known and unknown faces in the frame
    face_locations, face_names = sfr.detect_known_faces(frame)
    unknown_detected = False  # Flag to track if an unknown face is detected

    for face_loc, name in zip(face_locations, face_names):
        y1, x2, y2, x1 = face_loc
        cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)

        if name == "Unknown":
            unknown_detected = True
            current_time = time.time()

            # Check cooldown time before logging another unknown face
            if current_time - last_unknown_face_time >= cooldown_interval:
                cv2.putText(frame, "Unknown", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
                face_roi = frame[y1:y2, x1:x2]  # Extract face region of interest (ROI)
                unknown_faces_data.append(face_roi)  # Store the unknown face
                last_unknown_face_time = current_time

                # Play alert sound when an unknown face is detected
                print("⚠️ Unknown face detected! Playing alert sound...")
                threading.Thread(target=play_alert_sound, daemon=True).start()  

                # Start recording video when an unknown face is detected
                if not video_recording:
                    video_filename = os.path.join(video_folder, f"unknown_{int(time.time())}.avi")
                    fourcc = cv2.VideoWriter_fourcc(*'XVID')
                    fps = 20
                    frame_size = (frame.shape[1], frame.shape[0])
                    video_writer = cv2.VideoWriter(video_filename, fourcc, fps, frame_size)
                    video_recording = True
                    record_start_time = current_time
                    print(f"🔴 Recording started: {video_filename}")
        else:
            # Display the name of the recognized person
            cv2.putText(frame, name, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)

    # Save frames to video if recording is active
    if video_recording:
        video_writer.write(frame)
        if time.time() - record_start_time > 5:
            video_recording = False
            video_writer.release()
            print(f"🛑 Recording saved.")

    # Display the processed video frame
    cv2.imshow("Frame", frame)

    # Exit the loop if 'q' is pressed
    key = cv2.waitKey(1)
    if key & 0xFF == ord('q'):
        break

# Release resources when exiting
cap.release()
cv2.destroyAllWindows()

# --- Post-Processing: Handling Unknown Faces After Camera Closes ---
print("\nProcessing Unknown Faces after camera closed...")

if unknown_faces_data:
    for i, unknown_face_image in enumerate(unknown_faces_data):
        unknown_face_window_name = f"Unknown Face {i+1}"
        cv2.imshow(unknown_face_window_name, unknown_face_image)
        cv2.waitKey(0)
        
        # Prompt the user to identify the unknown face
        name_input = input(f"Identify Unknown Face {i+1}? Enter name (or type 'skip'): ")
        
        if name_input.lower() != 'skip':
            new_face_name = name_input.strip()
            if new_face_name:
                new_face_filename = os.path.join(faces_folder, f"{new_face_name}.jpg")
                cv2.imwrite(new_face_filename, unknown_face_image)
                print(f"Photo saved as {new_face_filename}")

                # Reload known face encodings after adding a new face
                try:
                    sfr.load_encoding_images(faces_folder)
                    print("Face encodings updated.")
                except Exception as e:
                    print(f"Error reloading face encodings: {e}")
            else:
                print("Name cannot be empty. Photo not saved.")
        else:
            print(f"Skipped saving Unknown Face {i+1}.")
        
        cv2.destroyWindow(unknown_face_window_name)
else:
    print("No unknown faces to process.")

print("Processing complete.")


4 encoding images found.
Encoding images loaded
⚠️ Unknown face detected! Playing alert sound...
🔴 Recording started: unknown_faces_videos\unknown_1742085868.avi
⚠️ Unknown face detected! Playing alert sound...
🛑 Recording saved.

Processing Unknown Faces after camera closed...
Name cannot be empty. Photo not saved.
Name cannot be empty. Photo not saved.
Processing complete.
